In [1]:
!ls


 AzureML			  julia
 MMLSpark			  kaggle.json
 SparkML			  pytorch
'Untitled Folder'		  seg_pred
 Untitled.ipynb			  seg_test
 Untitled1.ipynb		  seg_train
 catboost			  usamafile.ipynb
 h2o				  vgg_transfer_trained_wts.h5
 intel-image-classification.zip


In [2]:
import os
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from tqdm import tqdm
def load_data(data_dir):
  data = []
  labels = []
  class_dirs = os.listdir(data_dir)
  
  for direc in class_dirs:
    class_dir = os.path.join(data_dir, direc)
    for imagepath in tqdm(list(paths.list_images(class_dir))):
      image = cv2.imread(imagepath)
      image = cv2.resize(image, (150, 150))  # incase images not of same size
      data.append(image)
      labels.append(direc)
  # normalizing and converting to numpy array format
  data = np.array(data, dtype='float')/255.0
  labels = np.array(labels)
  return data, labels

In [3]:
train_dir = "seg_train/seg_train"
test_dir = "seg_test/"
pred_dir = "data/intel-image-classification-mini/seg_pred/"

print('loading train images')
X_train, y_train = load_data(train_dir)

print('loading test images')
X_test, y_test = load_data(test_dir)

  5%|▍         | 108/2271 [00:00<00:02, 1076.03it/s]

loading train images


100%|██████████| 2191/2191 [00:02<00:00, 1044.98it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
  3%|▎         | 100/3000 [00:00<00:02, 995.48it/s]

loading test images


100%|██████████| 3000/3000 [00:02<00:00, 1178.31it/s]


In [4]:
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.fit_transform(y_test)

In [5]:
import keras
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout
from keras.models import Sequential


vgg16 = Sequential()
vgg16.add(Conv2D(64, kernel_size=3, padding='same', activation='relu',
                 input_shape=(150, 150, 3)))
vgg16.add(Conv2D(64, kernel_size=3, padding='same', activation='relu'))
vgg16.add(MaxPooling2D(pool_size=2, strides=2))
vgg16.add(Dropout(0.25))

vgg16.add(Conv2D(128, kernel_size=3, padding='same', activation='relu'))
vgg16.add(Conv2D(128, kernel_size=3, padding='same', activation='relu'))
vgg16.add(MaxPooling2D(pool_size=2, strides=2))
vgg16.add(Dropout(0.25))

vgg16.add(Conv2D(256, kernel_size=3, padding='same', activation='relu'))
vgg16.add(Conv2D(256, kernel_size=3, padding='same', activation='relu'))
vgg16.add(Conv2D(256, kernel_size=3, padding='same', activation='relu'))
vgg16.add(MaxPooling2D(pool_size=2, strides=2))
vgg16.add(Dropout(0.25))

vgg16.add(Conv2D(512, kernel_size=3, padding='same', activation='relu'))
vgg16.add(Conv2D(512, kernel_size=3, padding='same', activation='relu'))
vgg16.add(Conv2D(512, kernel_size=3, padding='same', activation='relu'))
vgg16.add(MaxPooling2D(pool_size=2, strides=2))
vgg16.add(Dropout(0.25))

vgg16.add(Conv2D(512, kernel_size=3, activation='relu'))
vgg16.add(Conv2D(512, kernel_size=3, activation='relu'))
vgg16.add(Conv2D(512, kernel_size=3, activation='relu'))
vgg16.add(MaxPooling2D(pool_size=2, strides=2))
vgg16.add(Dropout(0.25))

vgg16.add(Flatten())

vgg16.add(Dense(4096, activation='relu'))
vgg16.add(Dense(4096, activation='relu'))
vgg16.add(Dense(4096, activation='relu'))

# the data we are using has 3 classes
vgg16.add(Dense(6, activation='softmax'))


In [6]:
vgg16.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 128)       0

In [7]:
from sklearn.model_selection import train_test_split
(X_train, X_valid, y_train, y_valid)= train_test_split(X_train, y_train, test_size=0.2, random_state=42)

from keras.optimizers import SGD
sgd = SGD(lr=0.001, decay=1e-7, momentum=.9)
vgg16.compile(loss='categorical_crossentropy',  optimizer=sgd,  metrics=['accuracy'])

In [8]:
H = vgg16.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
88/88 [==============================] - 3021s 34s/step - loss: 1.7916 - accuracy: 0.1815 - val_loss: 1.7911 - val_accuracy: 0.1853
Epoch 2/10
88/88 [==============================] - 2980s 34s/step - loss: 1.7913 - accuracy: 0.1731 - val_loss: 1.7908 - val_accuracy: 0.1853
Epoch 3/10
88/88 [==============================] - 3028s 34s/step - loss: 1.7908 - accuracy: 0.1753 - val_loss: 1.7905 - val_accuracy: 0.1853
Epoch 4/10
88/88 [==============================] - 3020s 34s/step - loss: 1.7902 - accuracy: 0.1822 - val_loss: 1.7904 - val_accuracy: 0.1853
Epoch 5/10
88/88 [==============================] - 3021s 34s/step - loss: 1.7899 - accuracy: 0.1784 - val_loss: 1.7902 - val_accuracy: 0.1853
Epoch 6/10
88/88 [==============================] - 3042s 35s/step - loss: 1.7894 - accuracy: 0.1719 - val_loss: 1.7900 - val_accuracy: 0.1853
Epoch 7/10
88/88 [==============================] - 1641s 18s/step - loss: 1.7894 - accuracy: 0.1737 - val_loss: 1.7898 - val_accuracy: 0.1853

In [10]:
#save the model's trained weights
vgg16.save_weights('vgg16_trained_10epochs.h5')

In [12]:
score = vgg16.evaluate(X_test, y_test, batch_size=128)
print('Test Loss = ', score[0])
print('Test Accuracy = ', score[1])

ValueError: in user code:

    /anaconda/envs/py37_default/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1233 test_function  *
        return step_function(self, iterator)
    /anaconda/envs/py37_default/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1224 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /anaconda/envs/py37_default/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /anaconda/envs/py37_default/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /anaconda/envs/py37_default/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /anaconda/envs/py37_default/lib/python